# Data Cleaning and Exploratory Data Analysis

* By - Gautam Sharma

# Import all required libraries

In [ ]:
import pandas as pd
!pip install missingno
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import zscore as zscore


# Data importing

In [ ]:
 data_lst = ['date_activ', 'date_end','date_first_activ','date_modif_prod', 'date_renewal']

In [ ]:
data_main = pd.read_csv('ml_case_training_data.csv', parse_dates=data_lst)
data_hist = pd.read_csv('ml_case_training_hist_data.csv', parse_dates=['price_date'])
data_output = pd.read_csv('ml_case_training_output.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data_main.head()

# I . Data Exploration
## The Output Dataset
* From thr output dataset we can derive a quick insights on customer retention.

In [ ]:
# Replace the churn columns
data_output['churn'] = data_output['churn'].replace({0:'Stayed',1:'Churned'})

In [ ]:
data_output.head(5)

In [ ]:
# What number of customers have  churned in the last 3 moths
attrition_count = data_output['churn'].value_counts()
print('Total number of churned customer : \n ', attrition_count)

* Last 3 moths 1595 customer have churned 
* currently 14501 actively client

In [ ]:
# Proportionof customer attrition in the last 3 months
attrition_rate = data_output['churn'].value_counts() / data_output.shape[0] * 100

print('Attrition rates in last 3 months: \n', attrition_rate)

* customer retenction in last 3 months 90.09%
* Customer attrition is 10 % in the last 3 months

# The History Dataset

In [ ]:
data_hist.head(5)

In [ ]:
data_hist1 = data_hist[['id','price_date']]

In [ ]:
data_hist1.head(5)

In [ ]:
# Examine the hist_data
data_hist1.info()

In [ ]:
data_hist.info()

In [ ]:
data_hist.describe()

In [ ]:
# Identify the nylity of the DataFrame
missing_values_hist = data_hist.isnull().sum()
print('Total missing data in data_hist: \n',missing_values_hist)

In [ ]:
# Identify the percentage of nullity in the dataframe for each column
missing_values_hist_per = data_hist.isnull().mean() * 100
print('The Total percentage of missing values: \n', missing_values_hist_per)

# The main Dataset

In [ ]:
data_main.head(5)

* The dataset contain more characteristics about each client'a account and activity.

In [ ]:
data_main.info()

In [ ]:
# Identify the percentage of nullity in the dataframe for each columns
missing_values_main_par = data_main.isnull().mean() * 100
print('Percentage of Missing values: \n', missing_values_main_par)

In [ ]:
# Examine the statistics of main dataset 
data_main.describe()

* The average net margin is $ 217
* The average num_years_antig is 5 years


# II. Data Cleaning and Imputation

* Here we dealing with missing dta and workflow for treating missing values

# The History Dataset

In [ ]:
data_hist.head(2)

In [ ]:
# Indentify the negative columns 
negative_col = ['price_p1_fix','price_p2_fix','price_p3_fix']

In [ ]:
data_hist[negative_col] = data_hist[negative_col].apply(abs)


In [ ]:
data_hist.describe()

## Visualizing the amount of missingness

In [ ]:
# Visualizethe completeness of the data frame
msno.bar(data_hist)

**As i see the data we absorb no data is missing but after the the imputation , we estimated that 0.7 % of data is missing**

In [ ]:
# Visua;ize the locations of the missing values of the dataset 
sorted = data_hist.sort_values(by = ['id', 'price_date'])
msno.matrix(sorted)

In [ ]:
# Visualize the correlation between the numeric variables of the Dataframe
msno.heatmap(data_hist)

In [ ]:
# Identify the index of the IDs containing missing values
hist_NAN_index = data_hist[data_hist.isnull().any(axis=1)].index.values.tolist()

In [ ]:
# Obtain the Dataframe with missing values
data_hist_missing = data_hist.iloc[hist_NAN_index,:]

In [ ]:
data_hist_missing.head(5)

In [ ]:
# Extract the unique dates of missing date
date_1st = data_hist_missing['price_date'].unique()
id_lst = data_hist_missing['id'].unique()

In [ ]:
# Create a time dataframe with the unique dates
time_data = pd.DataFrame(data=date_1st, columns=['price_date'])

In [ ]:
time_data.head(5)

* there are 1359 clients who are missing price data at least in 1 months

* There is hogh correlation between the missingness in the numeric and is values, missing or non-missing.

# Time series Data 

In [ ]:
# Make a copy of Data_hist dataset
data_hist_ff = data_hist.copy(deep=True)

In [ ]:
# Print prior to imputing missing values
print(data_hist_ff.iloc[hist_NAN_index,3:9].head())

In [ ]:
# FIll NAN using forward fill
data_hist_ff.fillna(method='ffill', inplace=True)

In [ ]:
print(data_hist_ff.iloc[hist_NAN_index,3:9].head(5))

In [ ]:
data_hist_ff.describe()

In [ ]:
# Merger output dataset with historical forwARD FILL DATASET
data_hist_ff_merged = data_hist_ff.merge(right=data_output, on=['id'])

In [ ]:
data_hist_ff_merged.head(5)

# The Main Dataset
**Visualizing the amountof missingness**

In [ ]:
# Visualize the completeness of the dataframe
msno.bar(data_main)

In [ ]:
# Visualize the locations of missing data
msno.matrix(data_main)

In [ ]:
sorted_main = data_main.sort_values('date_first_activ')

In [ ]:
msno.matrix(sorted_main)

In [ ]:
msno.heatmap(data_main)

In [ ]:
# Demonstrate why the date_activ column cannot replace completely date_first_activ
activity = ['date_activ','date_first_activ']

In [ ]:
# Filter the columns of interest
data_activity = data_main[activity]

In [ ]:
# Obtain only the complete interest
data_activity_cc = data_activity.dropna(subset=['date_first_activ'],how='any',inplace=False)

In [ ]:
# Test wether two objects contain the same elements
data_activity_cc.date_activ.equals(data_activity_cc.date_first_activ)

In [ ]:
# Describe the data
data_activity_cc.describe(datetime_is_numeric=True)

In [ ]:
# Drop the column activity_new and campaign_disc_elec
data_main_drop = data_main.drop(labels=['activity_new', 'campaign_disc_ele'],axis=1)

In [ ]:
# Remove date_end date_modif_prod date_renewal origin_up pow_max margin_gross_pow_ele margin_net_pow_ele net_margin
brush = ['date_end','date_modif_prod','date_renewal','origin_up','pow_max','margin_gross_pow_ele','margin_net_pow_ele', 'net_margin','forecast_discount_energy','forecast_price_energy_p1','forecast_price_energy_p2','forecast_price_pow_p1']

In [ ]:
data_main_drop.dropna(subset=brush, how='any', inplace=True)

In [ ]:
msno.matrix(data_main_drop)

In [ ]:
# Choose the columns without missing values
incomplete_cols = ['channel_sales','date_first_activ','forecast_base_bill_ele','forecast_base_bill_year','forecast_bill_12m','forecast_cons']

In [ ]:
complete_cols = [ column_name for column_name in data_main_drop.columns if column_name not in incomplete_cols]

In [ ]:
data_main_cc = data_main_drop[complete_cols]

In [ ]:
# Fix negative numeric variables
numeric = [column_name for column_name in data_main_cc.columns 
          if data_main_cc[column_name].dtype == 'float64' or data_main_cc[column_name].dtype == 'int64']

In [ ]:
# Overwrite positive values on negative values
data_main_cc[numeric] = data_main_cc[numeric].apply(abs)

In [ ]:
# Describe
data_main_cc.describe()

In [ ]:
# Convert the has_gas column to Yes/No
data_main_cc['has_gas'] = data_main_cc['has_gas'].replace({'t':'Yes', 'f':'No'})

In [ ]:
# Merger the main dataset withthe output dataset
data_main_cc_merged = data_main_cc.merge(right=data_output, on=['id'])

In [ ]:
# Convet the churn column to churned or stayed
data_main_cc_merged['churn'] = data_main_cc_merged['churn'].replace({1:'Churned',0:'Stayed'})

In [ ]:
data_main_cc_merged.head(5)

In [ ]:
# obtain all the variables except for id 
variables = [column_name for column_name in data_main_cc_merged.columns if column_name != 'id']

In [ ]:
#Obtain all the categorical variables except for id
categorical = [column_name for column_name in variables if data_main_cc_merged[column_name].dtype == 'object']

In [ ]:
# Obtain all the Data Variables
dates = [column_name for column_name in variables if data_main_cc_merged[column_name].dtype == 'datetime64[ns]']

In [ ]:
# Obtain all the numeric columns
numeric = [column_name for column_name in variables if column_name not in categorical and column_name != 'id' and 
          column_name != 'churn'
          and column_name not in dates]

# Data Visualization

* Let's visualize what we've found

In [ ]:
# Calculate the zcores of tenure
tenure_zcores = zscore(a=data_main_cc_merged['num_years_antig'])

In [ ]:
# Convert to absolute values
abs_tenure_zscores = np.abs(tenure_zcores)

In [ ]:
# Extract columns of intrest
churn_tenure = data_main_cc_merged[['churn','num_years_antig']]

In [ ]:
# Add z-score column
churn_tenure['z_score'] = list(abs_tenure_zscores)

In [ ]:
# Remove outliers
churned_tenure_filtered = churn_tenure[churn_tenure['z_score'] < 3]

In [ ]:
# Visualize tenure by retained customer and churner
vio = sns.violinplot( y=churned_tenure_filtered["churn"], x=churned_tenure_filtered["num_years_antig"] )
vio.set(xlabel = 'years', ylabel='')
vio.set_title('Customer Attrition by Tenure')
plt.show()

* Customer are more likely to churn during the 4th year that the 7th year
* The median age of retained customers is 5years
* The median age of churners is 4 years


# The Main Dataset


In [ ]:
# Most popular electrivcity cmpaign
elec_nm = data_main_cc_merged.loc[(data_main_cc_merged['churn'] >= 'Stayed')& (data_main_cc_merged['net_margin'] > 0), ['id','origin_up', 'net_margin']]

In [ ]:
elec_nm.value_counts(subset = ['origin_up'])

In [ ]:
# Highest netting electricity subscription campaign
print(elec_nm.groupby('origin_up')['net_margin'].agg('sum').sort_values(ascending=False))

In [ ]:
# Select current customers with positive net margins
top_customers = data_main_cc_merged.loc[(data_main_cc_merged['churn']>='Stayed') & (data_main_cc_merged['net_margin']>0),['id','num_years_antig','net_margin']]

# Top 10 customers by net margin
top_customers.sort_values(by=['net_margin'],ascending=False).head(10)

**These are the most profitable customers for PowerCo in terms of net margin. Beware most of them are within the likely tenure of attrition. Time for a marketing campaign!**